# Tabular data preprocessing

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.tabular import *
from fastai import *

## Overview

This package contains the basic class to define a transformation for preprocessing dataframes of tabular data, as well as basic [`TabularTransform`](/tabular.transform.html#TabularTransform). Preprocessing includes things like
- replacing non-numerical variables by categories, then their ids,
- filling missing values,
- normalizing continuous variables.

In all those steps we have to be careful to use the correspondance we decide on our training set (which id we give to each category, what is the value we put for missing data, or how the mean/std we use to normalize) on our validation or test set. To deal with this, we use a speciall class called [`TabularTransform`](/tabular.transform.html#TabularTransform).

The data used in this document page is a subset of the [adult dataset](https://archive.ics.uci.edu/ml/datasets/adult). It gives a certain amount of data on individuals to train a model to predict wether their salary is greater than \$50k or not.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')
train_df, valid_df = df.iloc[:800].copy(), df.iloc[800:1000].copy()
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


We see it contains numerical variables (like `age` or `education-num`) as well as categorical ones (like `workclass` or `relationship`). The original dataset is clean, but we removed a few values to give examples of dealing with missing variables.

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
cont_names = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

## Transforms for tabular data

In [ ]:
show_doc(TabularProc)

<h2 id="TabularProc"><code>class</code> <code>TabularProc</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L24" class="source_link">[source]</a></h2>

> <code>TabularProc</code>(`cat_names`:`StrList`, `cont_names`:`StrList`)

A processor for tabular dataframes.  

Base class for creating transforms for dataframes with categorical variables `cat_names` and continuous variables `cont_names`. Note that any column not in one of those lists won't be touched.

In [ ]:
show_doc(TabularProc.__call__)

<h4 id="TabularProc.__call__"><code>__call__</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L29" class="source_link">[source]</a></h4>

> <code>__call__</code>(`df`:`DataFrame`, `test`:`bool`=`False`)

Apply the correct function to `df` depending on `test`.  

In [ ]:
show_doc(TabularProc.apply_train)

<h4 id="TabularProc.apply_train"><code>apply_train</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L34" class="source_link">[source]</a></h4>

> <code>apply_train</code>(`df`:`DataFrame`)

Function applied to `df` if it's the train set.  

In [ ]:
show_doc(TabularProc.apply_test)

<h4 id="TabularProc.apply_test"><code>apply_test</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L37" class="source_link">[source]</a></h4>

> <code>apply_test</code>(`df`:`DataFrame`)

Function applied to `df` if it's the test set.  

In [ ]:
jekyll_important("Those two functions must be implemented in a subclass. `apply_test` defaults to `apply_train`.")

<div markdown="span" class="alert alert-warning" role="alert"><i class="fa fa-warning-circle"></i> <b>Important: </b>Those two functions must be implemented in a subclass. `apply_test` defaults to `apply_train`.</div>

The following [`TabularTransform`](/tabular.transform.html#TabularTransform) are implemented in the fastai library. Note that the replacement from categories to codes as well as the normalization of continuous variables are automatically done in a [`TabularDataset`](/tabular.data.html#TabularDataset).

In [ ]:
show_doc(Categorify)

<h2 id="Categorify"><code>class</code> <code>Categorify</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L41" class="source_link">[source]</a></h2>

> <code>Categorify</code>(`cat_names`:`StrList`, `cont_names`:`StrList`) :: [`TabularProc`](/tabular.transform.html#TabularProc)

Transform the categorical variables to that type.  

Variables in `cont_names` aren't affected.

In [ ]:
show_doc(Categorify.apply_train)

<h4 id="Categorify.apply_train"><code>apply_train</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L43" class="source_link">[source]</a></h4>

> <code>apply_train</code>(`df`:`DataFrame`)

Transform `self.cat_names` columns in categorical.  

In [ ]:
show_doc(Categorify.apply_test)

<h4 id="Categorify.apply_test"><code>apply_test</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L50" class="source_link">[source]</a></h4>

> <code>apply_test</code>(`df`:`DataFrame`)

Transform `self.cat_names` columns in categorical using the codes decided in `apply_train`.  

In [ ]:
tfm = Categorify(cat_names, cont_names)
tfm(train_df)
tfm(valid_df, test=True)

Since we haven't changed the categories by their codes, nothing visible has changed in the dataframe yet, but we can check that the variables are now categorical and view their corresponding codes.

In [ ]:
train_df['workclass'].cat.categories

Index([' ?', ' Federal-gov', ' Local-gov', ' Private', ' Self-emp-inc',
       ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
      dtype='object')

The test set will be given the same category codes as the training set.

In [ ]:
valid_df['workclass'].cat.categories

Index([' ?', ' Federal-gov', ' Local-gov', ' Private', ' Self-emp-inc',
       ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
      dtype='object')

In [ ]:
show_doc(FillMissing)

<h2 id="FillMissing"><code>class</code> <code>FillMissing</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L58" class="source_link">[source]</a></h2>

> <code>FillMissing</code>(`cat_names`:`StrList`, `cont_names`:`StrList`, `fill_strategy`:[`FillStrategy`](/tabular.transform.html#FillStrategy)=`<FillStrategy.MEDIAN: 1>`, `add_col`:`bool`=`True`, `fill_val`:`float`=`0.0`) :: [`TabularProc`](/tabular.transform.html#TabularProc)

Fill the missing values in continuous columns.  

`cat_names` variables are left untouched (their missing value will be repalced by code 0 in the [`TabularDataset`](/tabular.data.html#TabularDataset)). [`fill_strategy`](#FillStrategy) is adopted to replace those nans and if `add_col` is True, whenever a column `c` has missing values, a column named `c_nan` is added and flags the line where the value was missing.

In [ ]:
show_doc(FillMissing.apply_train)

<h4 id="FillMissing.apply_train"><code>apply_train</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L63" class="source_link">[source]</a></h4>

> <code>apply_train</code>(`df`:`DataFrame`)

Fill missing values in `self.cont_names` according to `self.fill_strategy`.  

In [ ]:
show_doc(FillMissing.apply_test)

<h4 id="FillMissing.apply_test"><code>apply_test</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L77" class="source_link">[source]</a></h4>

> <code>apply_test</code>(`df`:`DataFrame`)

Fill missing values in `self.cont_names` like in `apply_train`.  

Fills the missing values in the `cont_names` columns with the ones picked during train.

In [ ]:
train_df[cont_names].head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,49,101320,12.0,0,1902,40
1,44,236746,14.0,10520,0,45
2,38,96185,NaN,0,0,32
3,38,112847,15.0,0,0,40
4,42,82297,NaN,0,0,50


In [ ]:
tfm = FillMissing(cat_names, cont_names)
tfm(train_df)
tfm(valid_df, test=True)
train_df[cont_names].head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,49,101320,12.0,0,1902,40
1,44,236746,14.0,10520,0,45
2,38,96185,10.0,0,0,32
3,38,112847,15.0,0,0,40
4,42,82297,10.0,0,0,50


Values issing in the `education-num` column are replaced by 10, which is the median of the column in `train_df`. Categorical variables are not changed, since `nan` is simply used as another category.

In [ ]:
valid_df[cont_names].head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
800,45,96975,10.0,0,0,40
801,46,192779,10.0,15024,0,60
802,36,376455,10.0,0,0,38
803,25,50053,10.0,0,0,45
804,37,164526,10.0,0,0,40


In [ ]:
show_doc(FillStrategy, alt_doc_string='Enum flag represents determines how `FillMissing` should handle missing/nan values', arg_comments={
    'MEDIAN':'nans are replaced by the median value of the column',
    'COMMON': 'nans are replaced by the most common value of the column',
    'CONSTANT': 'nans are replaced by `fill_val`'
})

<h2 id="FillStrategy">`FillStrategy`</h2>

> <code>Enum</code> = [MEDIAN, COMMON, CONSTANT]

Enum flag represents determines how [`FillMissing`](/tabular.transform.html#FillMissing) should handle missing/nan values

- *MEDIAN*: nans are replaced by the median value of the column
- *COMMON*: nans are replaced by the most common value of the column
- *CONSTANT*: nans are replaced by `fill_val` 

In [ ]:
show_doc(Normalize)

<h2 id="Normalize"><code>class</code> <code>Normalize</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L89" class="source_link">[source]</a></h2>

> <code>Normalize</code>(`cat_names`:`StrList`, `cont_names`:`StrList`) :: [`TabularProc`](/tabular.transform.html#TabularProc)

Normalize the continuous variables.  

In [ ]:
show_doc(Normalize.apply_train)

<h4 id="Normalize.apply_train"><code>apply_train</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L91" class="source_link">[source]</a></h4>

> <code>apply_train</code>(`df`:`DataFrame`)

Comput the means and stds of `self.cont_names` columns to normalize them.  

In [ ]:
show_doc(Normalize.apply_test)

<h4 id="Normalize.apply_test"><code>apply_test</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L98" class="source_link">[source]</a></h4>

> <code>apply_test</code>(`df`:`DataFrame`)

Normalize `self.cont_names` with the same statistics as in `apply_train`.  

## Treating date columns

In [ ]:
show_doc(add_datepart)

<h4 id="add_datepart"><code>add_datepart</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L6" class="source_link">[source]</a></h4>

> <code>add_datepart</code>(`df`, `fldname`, `drop`=`True`, `time`=`False`)

Helper function that adds columns relevant to a date in the column `fldname` of `df`.  

Will `drop` the column in `df` if the flag is `True`. The `time` flag decides if we go down to the time parts or stick to the date parts.

## Undocumented Methods - Methods moved below this line will intentionally be hidden

## New Methods - Please document or move to the undocumented section

In [ ]:
show_doc(add_datepart)

<h4 id="add_datepart"><code>add_datepart</code><a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L6" class="source_link">[source]</a></h4>

> <code>add_datepart</code>(`df`, `fldname`, `drop`=`True`, `time`=`False`)

Helper function that adds columns relevant to a date in the column `fldname` of `df`.  